# EMICS 2021 Analysis (ROC analysis, 1kHz)

This R-notebook summarizes the statistical analyses performed to compare classification performances using their ROC curves, here using the 1kHz upsampled eye tracker data.

In [2]:
library(pROC)
library(jsonlite)

devices <- c('Eyelink', 'EyeSeeCam', 'EyeTribe')
classifiers <- c('LogR', 'kNN', 'SVM')

folder.in <- 'results/1000Hz'
file.in <- 'predictions.csv'

pred <- read.csv(file.path(folder.in, file.in), sep='\t')

## AUC statistics - slope method - Fig. 1 (right)

### AUC and 95% confidence intervals using all available responses

In [3]:
# Eye tracker comparison - 95% CI for AUC using the Stoll (2013) method
# Shown as the black markers in Fig. 1 (right)
pred.stoll <- pred[pred$model == 'Stoll2013',]

rocs.stoll <- list()
rocs.stoll.df <- data.frame();

for(tracker in devices) {

    mroc <- roc(pred.stoll$true_val[pred.stoll$device == tracker], 
                pred.stoll$slope[pred.stoll$device == tracker], 
                levels=c(1, 2), 
                dir='<')

    rocs.stoll[[tracker]] <- mroc
    ci.stoll <- as.numeric(ci.auc(mroc))
    rocs.stoll.df <- rbind.data.frame(rocs.stoll.df, c(ci.stoll))
}

rocs.stoll.df <- cbind.data.frame(devices, rocs.stoll.df)
colnames(rocs.stoll.df) <- c('device', 'ci.min', 'auc', 'ci.max')

# Save results (to be read by Python figure1() function)
write_json(rocs.stoll.df, file.path(folder.in, 'stoll_auc.json'))

# Print results to notebook
rocs.stoll.df

device,ci.min,auc,ci.max
<chr>,<dbl>,<dbl>,<dbl>
Eyelink,0.7203561,0.7714711,0.8225860
EyeSeeCam,0.8470491,0.8826531,0.9182570
EyeTribe,0.7087779,0.7595309,0.8102839


### Is performance similar between eye trackers when using the slope method?

In [4]:
# Eye tracker comparison - global ROCs
roc.EL_ES <- roc.test(rocs.stoll[['EyeSeeCam']], rocs.stoll[['Eyelink']])
print(roc.EL_ES)

roc.EL_ET <- roc.test(rocs.stoll[['Eyelink']], rocs.stoll[['EyeTribe']])
print(roc.EL_ET)

roc.ES_ET <- roc.test(rocs.stoll[['EyeSeeCam']], rocs.stoll[['EyeTribe']])
print(roc.ES_ET)

# Multiple comparisons correction
roc.p <- c(roc.EL_ES$p.value, roc.EL_ET$p.value, roc.ES_ET$p.value)
p.adjust(roc.p, method='holm')


	DeLong's test for two ROC curves

data:  rocs.stoll[["EyeSeeCam"]] and rocs.stoll[["Eyelink"]]
D = 3.4982, df = 598.13, p-value = 0.0005032
alternative hypothesis: true difference in AUC is not equal to 0
sample estimates:
AUC of roc1 AUC of roc2 
  0.8826531   0.7714711 


	DeLong's test for two ROC curves

data:  rocs.stoll[["Eyelink"]] and rocs.stoll[["EyeTribe"]]
D = 0.32489, df = 669.97, p-value = 0.7454
alternative hypothesis: true difference in AUC is not equal to 0
sample estimates:
AUC of roc1 AUC of roc2 
  0.7714711   0.7595309 


	DeLong's test for two ROC curves

data:  rocs.stoll[["EyeSeeCam"]] and rocs.stoll[["EyeTribe"]]
D = 3.8924, df = 600.44, p-value = 0.0001104
alternative hypothesis: true difference in AUC is not equal to 0
sample estimates:
AUC of roc1 AUC of roc2 
  0.8826531   0.7595309 



[1] 0.0010063592 0.7453670605 0.0003310969

### How many participants are significantly above chance level?

In [5]:
# Eyelink: individual subjects
subEL <- data.frame()
for(pp in 1:24) {
    mroc <- roc(pred.stoll$true_val[pred.stoll$device == 'Eyelink' & pred.stoll$ppid == pp], 
              pred.stoll$slope[pred.stoll$device == 'Eyelink' & pred.stoll$ppid == pp],
              levels=c(1, 2), 
              dir='<')
    auc <- auc(mroc)
    aci <- ci.auc(mroc)
    subEL <- rbind(subEL, c(pp, aci[1], aci[2], aci[3]))
}
colnames(subEL) <- c('pp', 'ci.min', 'auc', 'ci.max')
subEL$abovechance <- 0
subEL$abovechance[subEL$ci.min > 0.5] <- 1


# EyeSeeCam: individual subjects
subES <- data.frame()
for(pp in 1:24) {
    mroc <- roc(pred.stoll$true_val[pred.stoll$device == 'EyeSeeCam' & pred.stoll$ppid == pp], 
              pred.stoll$slope[pred.stoll$device == 'EyeSeeCam' & pred.stoll$ppid == pp],
              levels=c(1, 2), 
              dir='<')
    auc <- auc(mroc)
    aci <- ci.auc(mroc)
    subES <- rbind(subES, c(pp, aci[1], aci[2], aci[3]))
}
colnames(subES) <- c('pp', 'ci.min', 'auc', 'ci.max')
subES$abovechance <- 0
subES$abovechance[subES$ci.min > 0.5] <- 1


# EyeTribe: individual subjects
subET <- data.frame()
for(pp in 1:24) {
    mroc <- roc(pred.stoll$true_val[pred.stoll$device == 'EyeTribe' & pred.stoll$ppid == pp], 
              pred.stoll$slope[pred.stoll$device == 'EyeTribe' & pred.stoll$ppid == pp],
              levels=c(1, 2), 
              dir='<')
    auc <- auc(mroc)
    aci <- ci.auc(mroc)
    subET <- rbind(subET, c(pp, aci[1], aci[2], aci[3]))
}
colnames(subET) <- c('pp', 'ci.min', 'auc', 'ci.max')
subET$abovechance <- 0
subET$abovechance[subET$ci.min > 0.5] <- 1

cat('Participants classified above chance level:\n')
cat(paste('Eyelink:\tAUC > chance: ', sum(subEL$auc > 0.5), ', CImin > chance: ', sum(subEL$abovechance), '\n', sep=''))
cat(paste('EyeSeeCam:\tAUC > chance: ', sum(subES$auc > 0.5), ', CImin > chance: ', sum(subES$abovechance), '\n', sep=''))
cat(paste('EyeTribe:\tAUC > chance: ', sum(subET$auc > 0.5), ', CImin > chance: ', sum(subET$abovechance), '\n', sep=''))

Warning message in ci.auc.roc(mroc):
"ci.auc() of a ROC curve with AUC == 1 is always 1-1 and can be misleading."
Warning message in ci.auc.roc(mroc):
"ci.auc() of a ROC curve with AUC == 1 is always 1-1 and can be misleading."
Warning message in ci.auc.roc(mroc):
"ci.auc() of a ROC curve with AUC == 1 is always 1-1 and can be misleading."
Warning message in ci.auc.roc(mroc):
"ci.auc() of a ROC curve with AUC == 1 is always 1-1 and can be misleading."
Warning message in ci.auc.roc(mroc):
"ci.auc() of a ROC curve with AUC == 1 is always 1-1 and can be misleading."
Warning message in ci.auc.roc(mroc):
"ci.auc() of a ROC curve with AUC == 1 is always 1-1 and can be misleading."
Warning message in ci.auc.roc(mroc):
"ci.auc() of a ROC curve with AUC == 1 is always 1-1 and can be misleading."
Warning message in ci.auc.roc(mroc):
"ci.auc() of a ROC curve with AUC == 1 is always 1-1 and can be misleading."
Warning message in ci.auc.roc(mroc):
"ci.auc() of a ROC curve with AUC == 1 is always 1-

Participants classified above chance level:
Eyelink:	AUC > chance: 23, CImin > chance: 12
EyeSeeCam:	AUC > chance: 23, CImin > chance: 19
EyeTribe:	AUC > chance: 21, CImin > chance: 13


## Trial-based classification

### Which models differ from each other and from the Stoll et al. (2013) models?

In [6]:
pred.tr <- pred[pred$model %in% classifiers & pred$trace == 'trials' ,]

# Eye tracker comparison per classifier, using 95% CI for AUC
rocs.trial <- list()
for(tracker in devices) {
  
    rocs.trial[[tracker]] <- list()
  
    for (model in classifiers) {
        mroc <- roc(pred.tr$true_val[pred.tr$device == tracker & pred.tr$model == model], 
                    pred.tr$pred_prob[pred.tr$device == tracker & pred.tr$model == model],
              levels=c(1, 2), 
              dir='<')
        rocs.trial[[tracker]][[model]] <- mroc
    }
}

# Comparison to slope method when using full 10s trace (DS/full in Fig. 2)
pred.stoll.all <- pred[pred$model == 'Stoll_all',]
rocs.stoll.all <- list()
for(tracker in devices) {
    mroc <- roc(pred.stoll.all$true_val[pred.stoll.all$device == tracker], 
                pred.stoll.all$slope[pred.stoll.all$device == tracker], 
                levels=c(1, 2), 
                dir='<')
    rocs.stoll.all[[tracker]] <- mroc
}

# Compare all models using roc.test
final.trials <- data.frame()
for(tracker in devices) {
    for (model1 in classifiers) {
        for (model2 in classifiers) {
            roc.cmp <- roc.test(rocs.trial[[tracker]][[model1]], rocs.trial[[tracker]][[model2]])
            final.trials <- rbind(final.trials, c(tracker, model1, model2, roc.cmp$p.value[1], roc.cmp$statistic[1], roc.cmp$roc1$auc[1], roc.cmp$roc2$auc[1]))
        }

        # Also compare against Stoll2013 model...
        roc.cmpDS <- roc.test(rocs.trial[[tracker]][[model1]], rocs.stoll[[tracker]])
        final.trials <- rbind(final.trials, c(tracker, model1, 'DS', roc.cmpDS$p.value[1], roc.cmpDS$statistic[1], roc.cmpDS$roc1$auc[1], roc.cmpDS$roc2$auc[1]))

        # ...and DS_full model
        roc.cmpDSf <- roc.test(rocs.trial[[tracker]][[model1]], rocs.stoll.all[[tracker]])
        final.trials <- rbind(final.trials, c(tracker, model1, 'DS_full', roc.cmpDSf$p.value[1], roc.cmpDSf$statistic[1], roc.cmpDSf$roc1$auc[1], roc.cmpDSf$roc2$auc[1]))

    }
}
colnames(final.trials) <- c('tracker', 'model1', 'model2', 'p', 'z', 'auc1', 'auc2')

# Multiple comparisons correction, within each tracker
for(tracker in c('Eyelink', 'EyeSeeCam', 'EyeTribe')) {
  final.trials$p.holm[final.trials$tracker == tracker] <- p.adjust(final.trials$p[final.trials$tracker == tracker], method='holm')
}
final.trials$different <- 0
final.trials$different[final.trials$p.holm < 0.05] <- 1
final.trials$significance <- ''
final.trials$significance[final.trials$p.holm < 0.05] <- '*'
final.trials$significance[final.trials$p.holm < 0.01] <- '**'
final.trials$significance[final.trials$p.holm < 0.001] <- '***'

# Drop self-comparisons
final.trials <- final.trials[final.trials$z > 0,]

# Save comparison results
write.table(final.trials, file.path(folder.in, 'cls_comp_trials.csv'), sep='\t', row.names=F)

# Show table of model comparisons
final.trials

,tracker,model1,model2,p,z,auc1,auc2,p.holm,different,significance
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>
2,Eyelink,LogR,kNN,0.592758682898157,0.534842628620859,0.871031746031746,0.865575396825397,1.000000e+00,0,
4,Eyelink,LogR,DS,6.65567988615113e-06,4.50441254794331,0.871031746031746,0.771471088435374,5.324544e-05,1,***
5,Eyelink,LogR,DS_full,1.8650735598478e-06,4.76752014021929,0.871031746031746,0.789930555555556,1.678566e-05,1,***
9,Eyelink,kNN,DS,0.000195817262270085,3.72435236805132,0.865575396825397,0.771471088435374,1.174904e-03,1,**
10,Eyelink,kNN,DS_full,9.66387865017246e-06,4.4245605735774,0.865575396825397,0.789930555555556,6.764715e-05,1,***
11,Eyelink,SVM,LogR,2.45964187316565e-09,5.96411452452465,0.963364512471655,0.871031746031746,2.705606e-08,1,***
12,Eyelink,SVM,kNN,2.76634312983617e-12,6.98911058636687,0.963364512471655,0.865575396825397,3.596246e-11,1,***
14,Eyelink,SVM,DS,1.48914080238393e-14,7.68846213057859,0.963364512471655,0.771471088435374,2.084797e-13,1,***
15,Eyelink,SVM,DS_full,1.04937616589072e-15,8.0209415968033,0.963364512471655,0.789930555555556,1.574064e-14,1,***


## Response-interval-based classification

In [7]:
pred.ri <- pred[pred$model %in% classifiers & pred$trace == 'intervals' ,]

# Eye tracker comparison per classifier, using 95% CI for AUC
rocs.interval <- list()
for(tracker in devices) {
  
    rocs.interval[[tracker]] <- list()
  
    for (model in classifiers) {
        
        mroc <- roc(pred.ri$true_val[pred.ri$device == tracker & pred.ri$model == model], 
                    pred.ri$pred_prob[pred.ri$device == tracker & pred.ri$model == model],
                    levels=c(0, 1), 
                    dir='<')
        rocs.interval[[tracker]][[model]] <- mroc
    }
}

# Compare all models using roc.test
final.intervals <- data.frame()
for(tracker in devices) {
    for (model1 in classifiers) {
        for (model2 in classifiers) {
            roc.cmp <- roc.test(rocs.interval[[tracker]][[model1]], rocs.interval[[tracker]][[model2]])
            final.intervals <- rbind(final.intervals, c(tracker, model1, model2, roc.cmp$p.value[1], roc.cmp$statistic[1], roc.cmp$roc1$auc[1], roc.cmp$roc2$auc[1]))
        }
    }
}
colnames(final.intervals) <- c('tracker', 'model1', 'model2', 'p', 'z', 'auc1', 'auc2')

# Multiple comparisons correction, within each tracker
for(tracker in devices) {
  final.intervals$p.holm[final.intervals$tracker == tracker] <- p.adjust(final.intervals$p[final.intervals$tracker == tracker], method='holm')
}
final.intervals$different <- 0
final.intervals$different[final.intervals$p.holm < 0.05] <- 1
final.intervals$significance <- ''
final.intervals$significance[final.intervals$p.holm < 0.05] <- '*'
final.intervals$significance[final.intervals$p.holm < 0.01] <- '**'
final.intervals$significance[final.intervals$p.holm < 0.001] <- '***'

# Drop self-comparisons
final.intervals <- final.intervals[final.intervals$z > 0,]

# Save comparison results
write.table(final.intervals, file.path(folder.in, 'cls_comp_intervals.csv'), sep='\t', row.names=F)

# Show table of model comparisons
final.intervals

,tracker,model1,model2,p,z,auc1,auc2,p.holm,different,significance
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>
2,Eyelink,LogR,kNN,0.526114799227282,0.633947937754349,0.775147037981859,0.770496740362812,1.000000e+00,0,
7,Eyelink,SVM,LogR,1.42191066784375e-08,5.67072912051694,0.827620110544218,0.775147037981859,9.953375e-08,1,***
8,Eyelink,SVM,kNN,4.98341683173617e-11,6.57143043392835,0.827620110544218,0.770496740362812,4.485075e-10,1,***
11,EyeSeeCam,LogR,kNN,0.0668323807741228,1.83279958591436,0.838647959183674,0.826893778344671,3.341619e-01,0,
16,EyeSeeCam,SVM,LogR,2.43730400652143e-07,5.16245621151101,0.868693310657596,0.838647959183674,1.706113e-06,1,***
17,EyeSeeCam,SVM,kNN,5.34764502597451e-09,5.83597644734218,0.868693310657596,0.826893778344671,4.812881e-08,1,***
22,EyeTribe,kNN,LogR,0.685007164129273,0.405639958825347,0.808150864512472,0.805821286848073,1.000000e+00,0,
25,EyeTribe,SVM,LogR,1.06914997343778e-05,4.40269382903332,0.842536493764172,0.805821286848073,9.622350e-05,1,***
26,EyeTribe,SVM,kNN,0.000131684532485992,3.82328666092752,0.842536493764172,0.808150864512472,9.217917e-04,1,***
